In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Maza\\Desktop\\vector_db_pipeline\\research'

In [3]:
os.chdir("../")

In [14]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    read_data_dir: Path
    STATUS_FILE: str
    SCHEMA: dict



In [15]:
from vector_db_pipeline.constants import *
from vector_db_pipeline.utils.common import read_yaml, create_directories
from vector_db_pipeline import logger
import pandas as pd

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        

    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            read_data_dir=config.read_data_dir,
            STATUS_FILE=config.STATUS_FILE,
            SCHEMA=schema
        )

        return data_validation_config
    
config = ConfigurationManager()
TEST = config.get_data_validation_config()

[2024-04-08 12:57:13,393: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-04-08 12:57:13,395: INFO: common: yaml file: schema.yaml loaded successfully:]
[2024-04-08 12:57:13,397: INFO: common: Directory already exists: artifacts/data_validation:]


In [18]:
import os
from vector_db_pipeline import logger
import pandas as pd

In [59]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.read_data_dir)
            all_cols = list(data.columns)
            

            all_schema = self.config.SCHEMA
            all_schema['id'] = 'str'
            all_schema['values'] = 'list'
            print(all_schema)
            print(all_cols)

            
            for col in all_cols:
                if col not in all_schema.keys():
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f"All columns present in data: {validation_status}\n")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f"All columns present in data: {validation_status}\n")

            return logger.info(f"All columns present in data: {validation_status}")
        
        except Exception as e:
            raise e
        
    def validate_unique_index(self)-> bool:
        try:
            data = pd.read_csv(self.config.read_data_dir)
            unique_id = len(data.id.unique()) == len(data)
            if unique_id:
             
                with open(self.config.STATUS_FILE, 'a') as f:
                    f.write(f"Unique ids : {unique_id}\n")
            else:
               
                with open(self.config.STATUS_FILE, 'a') as f:
                    f.write(f"Unique ids : {unique_id}\n")
            return logger.info(f"Unique ids : {unique_id}")
        
        except Exception as e:
            raise e



In [60]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
    data_validation.validate_unique_index()
except Exception as e:
    raise e

[2024-04-08 13:24:11,724: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-04-08 13:24:11,727: INFO: common: yaml file: schema.yaml loaded successfully:]
[2024-04-08 13:24:11,730: INFO: common: Directory already exists: artifacts/data_validation:]
{'date_scraped_timestamp': 'int', 'host': 'str', 'page_title': 'str', 'text': 'str', 'url': 'str', 'id': 'str', 'values': 'list'}
['id', 'values', 'text', 'host', 'page_title', 'url']
[2024-04-08 13:24:13,909: INFO: 3017753284: All columns present in data: True:]
[2024-04-08 13:24:16,053: INFO: 3017753284: Unique ids : True:]
